In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [2]:
def fix_image_to_show(img):
    image = img.copy()
    image = image.astype('float32')
    max_pixel = np.max(image)
    min_pixel = np.min(image)
    if max_pixel == min_pixel:
        image[:, :, :] = 127
        image = image.astype('uint8')
        return image
    m = 255 / (max_pixel - min_pixel)
    image = image * m - min_pixel * m
    image = image.astype('uint8')
    return image

def show_image(img, name=None, cmap=None):
    image = img.copy()
    image = fix_image_to_show(image)
    plt.imshow(image, vmin=0, vmax=255, cmap=cmap)
    plt.show()
    if name is not None :
        plt.imsave(name + '.jpg', image, cmap=cmap)

In [3]:
import os 
directory = 'resources/Data/Train'
files = os.listdir(directory)
images = []
class_list = []
for file in files:
    path = directory + '/' + file 
    image_names = os.listdir(path)
    for image_name in image_names: 
        image = cv2.imread(path + '/' + image_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        images.append(image)
        class_list.append(file)

In [4]:
def convert_image_size(images, size): 
    images_c = []
    for image in images: 
        image = cv2.resize(image, size)
        images_c.append(image)
    return images_c
        

In [5]:
def L1_distance(image1, image2): 
    abs_image = np.abs(image1 - image2)
    return np.sum(abs_image)

def L2_distance(image1, image2):
    dif_image = np.power(image1 - image2, 2)
    return np.sum(dif_image)


In [6]:
def predict_class_knn(images, image, k, distance_func):
    global class_list 
    neighbours = list(range(len(images)))
    neighbours.sort(key=lambda x: distance_func(images[x], image))
    knn_class_list = list(map(lambda x: class_list[x], neighbours))
    knn_class_list = knn_class_list[0: k]
    knn_class_list_c = knn_class_list.copy()
    knn_class_list.sort(key=lambda x: -knn_class_list_c.count(x))
    return knn_class_list[0]

In [7]:
import os 
directory = 'resources/Data/Test'
files = os.listdir(directory)
test_images = []
test_class_list = []
for file in files:
    path = directory + '/' + file 
    image_names = os.listdir(path)
    for image_name in image_names: 
        image = cv2.imread(path + '/' + image_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        test_images.append(image)
        test_class_list.append(file)
        
        

In [8]:
def predict_class(images, image, k, sz, d): 
    image = image.copy()
    images = images.copy()
    images = convert_image_size(images, (sz, sz))
    image = cv2.resize(image, (sz, sz))
    dis_func = L1_distance 
    if d == 1:
        dis_func = L2_distance 
        
    return predict_class_knn(images, image, k, dis_func)
    

In [ ]:
#brute force on k and size and distance function 
for k in range(1, 3, 2): 
    for sz in range(20, 21):
        for d in range(1, 2): 
            correct = 0
            for i in range(len(test_images)):
                image = test_images[i]
                predict = predict_class(images, image, k, sz, d)
                if predict == test_class_list[i]:
                    correct = correct + 1
            
            rate = (correct / len(test_images)) * 100
            print("rate :", k, sz, d , rate)